# SkyLab Grupo 7

## 1. Recopilar información de las estrellas

In [29]:
from astroquery.simbad import Simbad

custom_simbad = Simbad()
custom_simbad.add_votable_fields('ra', 'dec')

stars = ["Alpha CMa", "Beta CMa", "Delta CMa", "HIP 33977", "Epsilon CMa", "Sigma CMa", "Eta CMa"]

# Main Query
results = custom_simbad.query_objects(stars)
print(results[["main_id", "ra", "dec", "user_specified_id"]])


  main_id           ra                 dec         user_specified_id
                   deg                 deg                          
----------- ------------------ ------------------- -----------------
  * alf CMa 101.28715533333335  -16.71611586111111       Alpha CMa  
  * bet CMa  95.67493896958332 -17.955918708888888       Beta CMa   
  * del CMa 107.09785021416667  -26.39319957888889       Delta CMa  
* omi02 CMa 105.75613058900544 -23.833293974200288       HIP 33977  
  * eps CMa 104.65645315148348 -28.972086157360806       Epsilon CMa
  * sig CMa 105.42978350186792 -27.934830340443888       Sigma CMa  
  * eta CMa 111.02375950100142 -29.303105508471724       Eta CMa    


## 2. Calcular Distancias Angulares

In [82]:
from math import acos, sin, cos, radians, degrees

def angular_distance(ra_1_deg: float,
                     dec_1_deg: float,
                     ra_2_deg: float,
                     dec_2_deg: float) -> float:
    ra_1, dec_1, ra_2, dec_2 = map(radians, [ra_1_deg, dec_1_deg, ra_2_deg, dec_2_deg])
    res = acos((sin(dec_1) * sin(dec_2)) + (cos(dec_1) * cos(dec_2) * cos(ra_1 - ra_2)))
    return degrees(res)

In [85]:
compare = [['Alpha CMa', 'Epsilon CMa'],
            ['Delta CMa', 'Epsilon CMa'],
            ['Sigma CMa', 'Epsilon CMa'],
            ['Delta CMa', 'Sigma CMa'],
            ['Delta CMa', 'HIP 33977'],
            ['Delta CMa', 'Eta CMa']
           ]

ans = []

for pair in compare:
    star_1 = custom_simbad.query_object(pair[0])
    star_2 = custom_simbad.query_object(pair[1])

    ra_1 = star_1["ra"].data[0]
    dec_1 = star_1["dec"].data[0]

    ra_2 = star_2["ra"].data[0]
    dec_2 = star_2["dec"].data[0]

    distance = angular_distance(ra_1, dec_1, ra_2, dec_2)
    ans.append(float(distance))
    print(f"Angular distance between {pair[0]} and {pair[1]}: {distance}")
# print(ans)

Angular distance between Alpha CMa and Epsilon CMa: 12.640930815128032
Angular distance between Delta CMa and Epsilon CMa: 3.364974264376436
Angular distance between Sigma CMa and Epsilon CMa: 1.240225513882492
Angular distance between Delta CMa and Sigma CMa: 2.1398274964731963
Angular distance between Delta CMa and HIP 33977: 2.833487172605846
Angular distance between Delta CMa and Eta CMa: 4.528940607532971


In [127]:
compare_px = [1232.87, 336.15, 123.12, 214.77, 285.26, 479.03]

compare_deg_px = list(zip(ans, compare_px))
scale = [deg / px for deg, px in compare_deg_px] # deg/px
scale2 = [print(deg / px) for deg, px in compare_deg_px] # deg/px
avg_scale = sum(scale) / len(scale)

print(f'Scale: {scale} deg/px')
print(f'Avg scale: {avg_scale} deg/px')

0.010253255262215832
0.010010335458504941
0.010073306642970207
0.009963344491657103
0.009932998571849701
0.00945439869639265
Scale: [0.010253255262215832, 0.010010335458504941, 0.010073306642970207, 0.009963344491657103, 0.009932998571849701, 0.00945439869639265] deg/px
Avg scale: 0.009947939853931739 deg/px


In [87]:
def px2deg(px: float, scale: float) -> float:
    return px * scale

## Respuesta

In [89]:
from math import ceil

px_measurements = [125.93, 127.20, 128.40, 127.14]
px_measurements = [ceil(px) for px in px_measurements]
avg_movement = sum([px2deg(px, avg_scale) for px in px_measurements]) / len(px_measurements)

print(f'Avg movement: {avg_movement} deg')


Avg movement: 1.2708493163397796 deg


### Análisis

In [123]:
def min2minsec(time: float) -> tuple:
    minutes = int(time)
    seconds = (time - int(time)) * 60
    return f'{minutes} min {seconds: .0f} sec'

In [136]:
period = 23 * 60 * 60 + 56 * 60 + 4
w_earth = 360 / (period)
print(f'Earth width: {w_earth: .5f} deg/sec')

Earth width:  0.00418 deg/sec


In [137]:
shift_time = avg_movement / w_earth
error = 39 # 2nd photo 39 seconds late
print(f'Shift time: {shift_time: .5f} sec - {error} seg. error')
print(f'Shift time: {min2minsec((shift_time - error) / 60)}')

Shift time:  304.17072 sec - 39 seg. error
Shift time: 4 min  25 sec
